### Library used: 
Alibi is an open source Python library aimed at machine learning model inspection and interpretation.
### Input data
In this dataset, each entry represents a person who takes a credit by a bank. <br>
Each person is classified as good or bad credit risks according to the set of attributes.<br>
Length: 1000 entries<br>
The column names are:<br>
1. Age (numeric)<br>
2. Sex (text: male, female)<br>
3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)<br>
4. Housing (text: own, rent, or free)<br>
5. Saving accounts (text - little, moderate, quite rich, rich)<br>
6. Checking account (numeric)<br>
7. Credit amount (numeric)<br>
8. Duration (numeric, in month)<br>
9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

In [1]:
pip install alibi

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests<3.0.0,>=2.21.0 in c:\users\acer\anaconda3\lib\site-packages (from alibi) (2.25.1)



You should consider upgrading via the 'C:\Users\Acer\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from alibi.explainers import CounterfactualProto
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model,load_model
tf.get_logger().setLevel(40) # Only error will be printed, not warning or info messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs 
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from alibi.utils.mapping import ohe_to_ord, ord_to_ohe

In [3]:
df = pd.read_csv("german_credit_data.csv", index_col=0)
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [4]:
df.isnull().any()

Age                 False
Sex                 False
Job                 False
Housing             False
Saving accounts      True
Checking account     True
Credit amount       False
Duration            False
Purpose             False
Risk                False
dtype: bool

In [5]:
 def replace_missingValue_with_mode(dataFrame):
    
    columns_with_missing_value = dataFrame.columns[dataFrame.isnull().any()]
    
    for col in columns_with_missing_value:
        col_mode = dataFrame[col].mode()[0]
        dataFrame[col].fillna(col_mode, inplace=True)

In [6]:
replace_missingValue_with_mode(df)

In [7]:
df.isnull().any()

Age                 False
Sex                 False
Job                 False
Housing             False
Saving accounts     False
Checking account    False
Credit amount       False
Duration            False
Purpose             False
Risk                False
dtype: bool

In [8]:
# Label encoder: Encode target labels with value between 0 and n_classes-1

def categorical_to_numeric(df):
    labelencoder= LabelEncoder()
    for col in df.columns.to_list():
        if(df[col].dtype == "object"):
            print(col)
            print(df[col].value_counts())
            df[col] = labelencoder.fit_transform(df[col])
            print(df[col].value_counts())
    return df
df_new = categorical_to_numeric(df.copy())
df_new

Sex
male      690
female    310
Name: Sex, dtype: int64
1    690
0    310
Name: Sex, dtype: int64
Housing
own     713
rent    179
free    108
Name: Housing, dtype: int64
1    713
2    179
0    108
Name: Housing, dtype: int64
Saving accounts
little        786
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64
0    786
1    103
2     63
3     48
Name: Saving accounts, dtype: int64
Checking account
little      668
moderate    269
rich         63
Name: Checking account, dtype: int64
0    668
1    269
2     63
Name: Checking account, dtype: int64
Purpose
car                    337
radio/TV               280
furniture/equipment    181
business                97
education               59
repairs                 22
domestic appliances     12
vacation/others         12
Name: Purpose, dtype: int64
1    337
5    280
4    181
0     97
3     59
6     22
2     12
7     12
Name: Purpose, dtype: int64
Risk
good    700
bad     300
Name: Risk, dtype: int64
1    70

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,0,1169,6,5,1
1,22,0,2,1,0,1,5951,48,5,0
2,49,1,1,1,0,0,2096,12,3,1
3,45,1,2,0,0,0,7882,42,4,1
4,53,1,2,0,0,0,4870,24,1,0
...,...,...,...,...,...,...,...,...,...,...
995,31,0,1,1,0,0,1736,12,4,1
996,40,1,3,1,0,0,3857,30,1,1
997,38,1,2,1,0,0,804,12,5,1
998,23,1,2,0,0,0,1845,45,5,0


In [9]:
X = np.c_[df_new.iloc[:,1:6], df_new.iloc[:,8:9], df_new.iloc[:,0:1], df_new.iloc[:,6:8]]
X

array([[   1,    2,    1, ...,   67, 1169,    6],
       [   0,    2,    1, ...,   22, 5951,   48],
       [   1,    1,    1, ...,   49, 2096,   12],
       ...,
       [   1,    2,    1, ...,   38,  804,   12],
       [   1,    2,    0, ...,   23, 1845,   45],
       [   1,    2,    1, ...,   27, 4576,   45]], dtype=int64)

In [10]:
cate_dict={0: 2, 1: 4, 2: 3, 3: 4, 4: 3, 5: 8}
cat_vars_ohe = ord_to_ohe(X, cate_dict)
cat_vars_ohe = cat_vars_ohe[1]
print(cat_vars_ohe)

{0: 2, 2: 4, 6: 3, 9: 4, 13: 3, 16: 8}


In [11]:
X[:,:-3]

array([[1, 2, 1, 0, 0, 5],
       [0, 2, 1, 0, 1, 5],
       [1, 1, 1, 0, 0, 3],
       ...,
       [1, 2, 1, 0, 0, 5],
       [1, 2, 0, 0, 0, 5],
       [1, 2, 1, 1, 1, 1]], dtype=int64)

In [12]:
#Convert ordinal values to One Hot encoding
ohe = OneHotEncoder(categories='auto')
ohe.fit(X[:,:-3])
X_ohe = ohe.transform(X[:,:-3])

In [13]:
X_ohe

<1000x24 sparse matrix of type '<class 'numpy.float64'>'
	with 6000 stored elements in Compressed Sparse Row format>

In [14]:
X_ohe.todense()

matrix([[0., 1., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]])

In [15]:
#Standardize the numerical columns
x_num = pd.concat([df_new.iloc[:,0:1], df_new.iloc[:,6:8]], axis=1)
print(x_num)
mu = x_num.mean(axis=0)
print(mu)
sigma = x_num.std(axis=0)
print(sigma)
x_num= (x_num - mu) / sigma
x_num

     Age  Credit amount  Duration
0     67           1169         6
1     22           5951        48
2     49           2096        12
3     45           7882        42
4     53           4870        24
..   ...            ...       ...
995   31           1736        12
996   40           3857        30
997   38            804        12
998   23           1845        45
999   27           4576        45

[1000 rows x 3 columns]
Age                35.546
Credit amount    3271.258
Duration           20.903
dtype: float64
Age                11.375469
Credit amount    2822.736876
Duration           12.058814
dtype: float64


,Age,Credit amount,Duration
0,2.765073,-0.744759,-1.235859
1,-1.190808,0.949342,2.247070
2,1.182721,-0.416354,-0.738298
3,0.831087,1.633430,1.749509
4,1.534354,0.566380,0.256825
...,...,...,...
995,-0.399632,-0.543890,-0.738298
996,0.391544,0.207509,0.754386
997,0.215727,-0.874066,-0.738298
998,-1.102900,-0.505275,1.998289


In [16]:
#Concatenate the modified categorical and numerical columns
df_clean =  np.c_[X_ohe.todense(), x_num, df_new.iloc[:,-1:]]
df_clean

matrix([[ 0.        ,  1.        ,  0.        , ..., -0.74475875,
         -1.23585947,  1.        ],
        [ 1.        ,  0.        ,  0.        , ...,  0.94934176,
          2.24706998,  0.        ],
        [ 0.        ,  1.        ,  0.        , ..., -0.41635407,
         -0.73829812,  1.        ],
        ...,
        [ 0.        ,  1.        ,  0.        , ..., -0.87406588,
         -0.73829812,  1.        ],
        [ 0.        ,  1.        ,  0.        , ..., -0.50527487,
          1.99828931,  0.        ],
        [ 0.        ,  1.        ,  0.        , ...,  0.46222587,
          1.99828931,  1.        ]])

In [17]:
df_clean.shape

(1000, 28)

In [18]:
train_data, test_data = train_test_split(df_clean, test_size=0.2)

In [19]:
x_train,y_train=train_data[:,:-1],train_data[:,-1:]
x_test,y_test=test_data[:,:-1],test_data[:,-1:]

In [20]:
def nn_model():
    x_in = Input(shape=(27,))
    x = Dense(18, activation='relu')(x_in)
#     x = Dense(9, activation='softmax')(x)
    x_out_l = Dense(2, activation='sigmoid')(x)
    nn = Model(inputs=x_in, outputs=x_out_l)
    nn.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn
#the sigmoid is a function that only occupies the range from 0 to 1 and it asymptotes both values. This makes it very handy for binary classification with 0 and 1 as potential output values.

In [21]:
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=10, epochs=200, verbose=1)
#output_size * (input_size + 1) == number_parameters 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 27)]              0         
                                                                 
 dense (Dense)               (None, 18)                504       
                                                                 
 dense_1 (Dense)             (None, 2)                 38        
                                                                 
Total params: 542
Trainable params: 542
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples
Epoch 1/200
800/800 [==============================] - 0s 135us/sample - loss: 0.6126 - acc: 0.6900
Epoch 2/200
800/800 [==============================] - 0s 53us/sample - loss: 0.5994 - acc: 0.6875
Epoch 3/200
800/800 [==============================] - 0s 49us/sample - loss: 0.5936 - acc: 0.6938
Epoch 4/

In [22]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', score[1])

Test accuracy:  0.7


C:\Users\Acer\anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [23]:
for layer in nn.layers:
    print(layer.name, layer.inbound_nodes, layer.outbound_nodes)

input_1 [<keras.engine.node.Node object at 0x000001D86E34BDC0>] [<keras.engine.node.Node object at 0x000001D86E35D820>]
dense [<keras.engine.node.Node object at 0x000001D86E35D820>] [<keras.engine.node.Node object at 0x000001D86E35D610>]
dense_1 [<keras.engine.node.Node object at 0x000001D86E35D610>] []


In [24]:
names = [weight.name for layer in nn.layers for weight in nn.weights]
weights = nn.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)
for layer in nn.layers:
    print(layer,"----------",nn.get_weights())

dense/kernel:0 (27, 18)
dense/bias:0 (18,)
dense_1/kernel:0 (18, 2)
dense_1/bias:0 (2,)
<keras.engine.input_layer.InputLayer object at 0x000001D86E1E07F0> ---------- [array([[ 2.03999087e-01, -1.28053084e-01,  2.51313955e-01,
         2.17519537e-01, -3.85622919e-01, -2.66590148e-01,
        -6.35308474e-02,  3.64601277e-02, -9.65961143e-02,
         3.32777709e-01,  1.93458274e-01,  3.26159805e-01,
         9.57212746e-02, -6.75269067e-02,  2.37667665e-01,
        -2.67305911e-01,  9.46578905e-02, -1.99721649e-01],
       [-9.73574147e-02, -5.87192923e-02, -1.46250157e-02,
         3.52003157e-01,  3.36519897e-01, -2.04159990e-01,
        -3.80150437e-01,  3.19869280e-01, -8.93107131e-02,
        -1.98329724e-02,  2.38479704e-01, -3.18224043e-01,
         2.32561097e-01, -1.25210807e-01, -5.98012030e-01,
        -4.82303929e-03, -1.40538469e-01,  6.90590218e-02],
       [-2.53820360e-01,  1.98137179e-01,  1.41085058e-01,
        -1.87789813e-01, -2.18788221e-01, -3.34584951e-01,
     

CounterfactualProto parameters: <br>

* shape: shape of the instance to be explained<br>
* beta: loss term multiplier. A higher value means more weight on the sparsity restrictions of the perturbations.<br>
* theta: multiplier for the last loss term. A higher value means more emphasis on the gradients guiding the counterfactual towards the nearest class prototype. <br>
* c_init and c_steps: the multiplier of the first loss term is updated for *c_steps* iterations, starting at *c_init*<br>

d_type refers to the distance metric used to convert the categorical to numerical values. <br>
Option values: abdm, mvdm and abdm-mvdm. <br>
1. abdm: uses context provided by the other variables.<br>
2. mvdm: using the model predictions<br>
3. abdm-mvdm: combines both methods.<br>

In [27]:
shape = x_test[1].shape
a=(np.ones((1,5)) * 0)
b=(np.ones((1,5)) * 1)
c = np.min(df_clean[:,-4:], axis=0)
d = np.max(df_clean[:,-4:], axis=0)
min_feature = np.concatenate((a, c), axis=1)
max_feature = np.concatenate((b, d), axis=1)
feature_range = (min_feature.astype(np.float32),
                 max_feature.astype(np.float32))
feature_range

(matrix([[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          -1.4545335, -1.0703293, -1.4017133,  0.       ]], dtype=float32),
 matrix([[1.       , 1.       , 1.       , 1.       , 1.       ,
          3.4683406, 5.368103 , 4.237315 , 1.       ]], dtype=float32))

In [28]:
cf = CounterfactualProto(nn,
                         shape,
                         beta=0.01,
                         theta = 100,
                         cat_vars=cat_vars_ohe,
                         ohe=True,
                         max_iterations=500,
                         feature_range=feature_range,
                         c_init=1,
                         c_steps=5
                        )

C:\Users\Acer\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [29]:
cf.fit(x_train, d_type='mvdm');


In [37]:
nn.predict(x_test[7])

array([[0.5721996, 0.3574605]], dtype=float32)

In [38]:
nn.predict(x_test[4])

array([[0.09084836, 0.8194923 ]], dtype=float32)

In [45]:
x0 = x_test[7].reshape((1,) + x_test[7].shape)
explanation0 = cf.explain(x0)
x1 = x_test[4].reshape((1,) + x_test[4].shape)
explanation1 = cf.explain(x1)

In [46]:
explanation1

Explanation(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.01,
              'feature_range': (matrix([[ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         -1.4545335, -1.0703293, -1.4017133,  0.       ]], dtype=float32), matrix([[1.       , 1.       , 1.       , 1.       , 1.       ,
         3.4683406, 5.368103 , 4.237315 , 1.       ]], dtype=float32)),
              'gamma': 0.0,
              'theta': 100,
              'cat_vars': {0: 2, 2: 4, 6: 3, 9: 4, 13: 3, 16: 8},
              'ohe': True,
              'use_kdtree': False,
              'learning_rate_init': 0.01,
              'max_iterations': 500,
              'c_init': 1,
              'c_steps': 5,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'shape': (1

In [47]:
print(f'Original prediction of x0: {explanation0.orig_class}')
print('Counterfactual prediction of x0: {}'.format(explanation0.cf['class']))
print(f'Original prediction of x1: {explanation1.orig_class}')
print('Counterfactual prediction of x1: {}'.format(explanation1.cf['class']))

Original prediction of x0: 0
Counterfactual prediction of x0: 1
Original prediction of x1: 1
Counterfactual prediction of x1: 0


In [48]:
def describe_instance(x, explanation, eps=1):
    target_names = ['Good', 'Bad']
    feature_names = ['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account','Purpose', 'Age', 'Credit amount', 'Duration']
    print('Original instance: {}  -- probability: {}'.format(target_names[explanation.orig_class],
                                                       explanation.orig_proba[0]))
    print('Counterfactual instance: {}  -- probability: {}'.format(target_names[explanation.cf['class']],
                                                             explanation.cf['proba'][0]))
    category_map = {}
    i=0;
    for col in df.iloc[:,:-1].columns.to_list():
            if(i==1):
                category_map[i] = ['0','1','2','3'] #for job
                i=i+1
            if(df[col].dtype == "object"):
                category_map[i] = df[col].unique().tolist()
                i=i+1
    
    X = np.c_[df_new.iloc[:,1:6], df_new.iloc[:,8:9], df_new.iloc[:,0:1], df_new.iloc[:,6:8]]
    cat_vars_ord = {}
    n_categories = len(list(category_map.keys()))
    for i in range(n_categories):
        cat_vars_ord[i] = len(np.unique(X[:, i]))

    print('\nCounterfactual perturbations...')
    print('\nCategorical:')
    X_orig_ord = ohe_to_ord(x, cat_vars_ohe)[0]
    X_cf_ord = ohe_to_ord(explanation.cf['X'], cat_vars_ohe)[0]
    delta_cat = {}
    for i, (_, v) in enumerate(category_map.items()):
        cat_orig = v[int(X_orig_ord[0, i])]
        cat_cf = v[int(X_cf_ord[0, i])]
        if cat_orig != cat_cf:
            delta_cat[feature_names[i]] = [cat_orig, cat_cf]
    if delta_cat:
        for k, v in delta_cat.items():
            print('{}: {}  -->   {}'.format(k, v[0], v[1]))
    print('\nNumerical:')
    X_orig_ord = np.asmatrix(pd.DataFrame(x[:,-3:],columns=['Age', 'Credit amount', 'Duration'])* sigma + mu)
    X_cf_ord = np.asmatrix(pd.DataFrame(explanation.cf['X'][:,-3:],columns=['Age', 'Credit amount', 'Duration'])* sigma + mu)
    delta_num = (X_cf_ord) - (X_orig_ord)
    n_keys = len(list(cat_vars_ord.keys()))
    for i in range(delta_num.shape[1]):
        if np.abs(delta_num[0, i]) > eps:
            print('{}: {:.2f}  -->   {:.2f}'.format(feature_names[i+n_keys],
                                            X_orig_ord[0,i],
                                            X_cf_ord[0,i]))

In [49]:
describe_instance(x0, explanation0)
#  job values: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled

Original instance: Good  -- probability: [0.5721996 0.3574605]
Counterfactual instance: Bad  -- probability: [0.38731623 0.39596432]

Counterfactual perturbations...

Categorical:
Job: 1  -->   2
Checking account: moderate  -->   little
Purpose: radio/TV  -->   education

Numerical:
Credit amount: 5954.00  -->   6086.09
Duration: 42.00  -->   40.88


In [50]:
describe_instance(x1, explanation1)

Original instance: Bad  -- probability: [0.09084836 0.8194923 ]
Counterfactual instance: Good  -- probability: [0.4960399  0.46273115]

Counterfactual perturbations...

Categorical:
Job: 1  -->   3
Housing: free  -->   own
Purpose: domestic appliances  -->   business

Numerical:
Duration: 12.00  -->   20.61
